<a href="https://colab.research.google.com/gist/yiftachn/49329d278b380a86cfe2a92a4fe2dc76/hw3-sna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [154]:
import networkx as nx
import pandas as pd
import numpy as np
import bokeh as bk
import glob
from tqdm import tqdm
from networkx.algorithms import approximation as apx
path = "files/"
from sklearn.cluster import KMeans, DBSCAN, MeanShift
from collections import Counter

# load networks

In [38]:
networks = []
for file in glob.glob(path+"*.txt"):
  df = pd.read_csv(file,sep=" ",header = None)
  g = nx.from_pandas_edgelist(df,0,1)
  g.name = file[6:]
  networks.append(g)

In [30]:
for x in glob.glob(path+"*.txt"):
    print (x[6:])

as-caida20050404.ngraph.txt
as-caida20050905.ngraph.txt
as-caida20051205.ngraph.txt
as-caida20060102.ngraph.txt
as-caida20060306.ngraph.txt
as-caida20060327.ngraph.txt
as-caida20060515.ngraph.txt
as-caida20060619.ngraph.txt
as-caida20060724.ngraph.txt
as-Oregon1_010331.ungraph.txt
as-Oregon1_010407.ungraph.txt
as-Oregon1_010414.ungraph.txt
as-Oregon1_010421.ungraph.txt
as-Oregon1_010428.ungraph.txt
as-Oregon1_010505.ungraph.txt
as-Oregon1_010512.ungraph.txt
as-Oregon1_010519.ungraph.txt
as-Oregon1_010526.ungraph.txt
BAA1(1000).ungraph.txt
BAA1(1500).ungraph.txt
BAA1(500).ungraph.txt
BAA2(1000).ungraph.txt
BAA2(1500).ungraph.txt
BAA2(500).ungraph.txt
BAA3(1000).ungraph.txt
BAA3(1500).ungraph.txt
BAA3(500).ungraph.txt
cn-dblp (1218).ungraph.txt
cn-dblp (1328).txt
cn-dblp (1329).txt
cn-dblp (1330).txt
cn-dblp (1331).txt
cn-dblp (1332).txt
cn-dblp (1333).txt
cn-dblp (1334).txt
cn-dblp (1335).txt
ER1(1000).ungraph.txt
ER1(1500).ungraph.txt
ER1(500).ungraph.txt
ER2(1000).ungraph.txt
ER2(1500

# Calculate labes and divide train and test

In [33]:
network_labels = {}
for file in glob.glob(path+"*.txt"):
  name = file[6:]
  sep = name.find('-')
  if sep == -1 :
    network_labels[name] = name[0:2]
  else:
    network_labels[name] = name[:sep]

In [42]:
type({(x,1) for x in [1,2,3]})

set

# calculate features

In [43]:
network_features = pd.DataFrame(dict((x.name[6:],0) for x in networks),columns=['name','average_degree']).set_index('name')


## calculate average degree by std



In [44]:
for network in networks:
  degrees = []
  for d in nx.degree(network):
    degrees.append(d[1])
  degrees = np.array(degrees)
  std_by_mean = np.std(degrees)/np.mean(degrees)
  network_features.loc[network.name,'degree_mean'] = np.mean(degrees)
  network_features.loc[network.name,'degree_std_by_mean'] = std_by_mean

## calculate connected components number


In [45]:
for network in tqdm(networks):
  network_features.loc[network.name,'connected_components'] = nx.number_connected_components(network)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:02<00:00, 52.56it/s]


## calculate clustering coefficient

In [46]:
for network in tqdm(networks):
  network_features.loc[network.name,'avg_clustering_coefficient'] = apx.clustering_coefficient.average_clustering(network)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 73.76it/s]


## calculate vertex cover by size


In [47]:
for network in networks:
    network_features.loc[network.name,'vertex_cover_by_size'] = len(apx.min_weighted_vertex_cover(network))/ nx.number_of_nodes(network)


In [52]:
network_features = network_features.drop(columns = 'average_degree')

# inspect features

In [168]:
s = network_features.degree_std_by_mean
s = (s -s.mean())/s.std()
s.std()

0.9999999999999997

# train model

In [156]:
models = [KMeans(n_clusters=12),MeanShift(),DBSCAN()]
predictions = []
for model in models:
    predictions.append(model.fit_predict(network_features))

In [157]:
predtuples = []
for prediction in predictions:
    predtuples.append([(prediction[i],network_labels[network_features.iloc[i].name]) for i in range(len(network_features))])

In [148]:
def purity(predictions):
    predictions.sort()
    clusters = set(x for x,y in predictions)
    sum = 0
    for cluster in clusters:
        counter = Counter(y for x,y in predictions if x==cluster)
        sum += max(counter.values())
    return sum /len(predictions)

In [160]:
purity(predtuples[2])

0.46296296296296297

In [151]:
predtuples

[(0, 'ER'),
 (0, 'ER'),
 (0, 'ER'),
 (0, 'cn'),
 (0, 'ppi'),
 (0, 'ppi'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (0, 'sn'),
 (1, 'BA'),
 (1, 'BA'),
 (1, 'BA'),
 (2, 'p2p'),
 (2, 'p2p'),
 (2, 'p2p'),
 (2, 'p2p'),
 (3, 'BA'),
 (3, 'BA'),
 (3, 'BA'),
 (4, 'ppi'),
 (4, 'ppi'),
 (5, 'BA'),
 (5, 'BA'),
 (5, 'BA'),
 (6, 'ER'),
 (6, 'ER'),
 (6, 'ER'),
 (7, 'ppi'),
 (7, 'ppi'),
 (7, 'ppi'),
 (7, 'rn'),
 (7, 'rn'),
 (8, 'ER'),
 (8, 'ER'),
 (8, 'ER'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (9, 'as'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'WS'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'cn'),
 (10, 'p2p'),
 (10, 'p2p'),
 (10, 'p2p'),
 (10, 'p2p'),
 (10

In [153]:
for i in range (0,13):
    c = Counter(y for x,y in predtuples if x==i)
    print(c)

Counter({'sn': 9, 'ER': 3, 'ppi': 2, 'cn': 1})
Counter({'BA': 3})
Counter({'p2p': 4})
Counter({'BA': 3})
Counter({'ppi': 2})
Counter({'BA': 3})
Counter({'ER': 3})
Counter({'ppi': 3, 'rn': 2})
Counter({'ER': 3})
Counter({'as': 18})
Counter({'WS': 9, 'prod': 9, 'sn': 9, 'cn': 8, 'rn': 7, 'p2p': 5, 'ppi': 1})
Counter({'ppi': 1})
Counter()


# predict and evaluate